# 读取数据
数据集一共有20个csv文件均来自 QueryFormer_VLDB2022/data/imdb.   
这是以及处理好的训练数据集,但这个目前是csv格式.     
在这一步中,主要是读取对应的csv文件然后去除第一行标题,并且合并所有数据储存在df中.    
然后将数据分为 x 和 y, 其中 x 是 json 字符串, y 是 Execution Time.用于最后训练评估


In [1]:
# 读取数据
import glob
import pandas as pd
import json

# 匹配所有 train_plan_0*.csv
files = glob.glob("../data/train_plan_*.csv")
print("找到的文件:", files)

# 读入并合并
df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

print("总数据行数:", len(df))
print("df:\n", df.head())

#获取json字符串
plans_json = df['json']
print("plans_json:\n", plans_json.iloc[0])

#字符串转json
plans_dict = []
ExecutionTimes = []
idx = 0
for json_str in plans_json:
    idx += 1
    plan_dict = json.loads(json_str)
    plans_dict.append(plan_dict['Plan'])
    try:
        ExecutionTimes.append(plan_dict['Execution Time'])
    except:
        print(f"idx: {idx} 不存在Execution Time")
        print(plan_dict)
print("plans_dict:\n", plans_dict[0])



找到的文件: ['../data/train_plan_part17.csv', '../data/train_plan_part8.csv', '../data/train_plan_part6.csv', '../data/train_plan_part3.csv', '../data/train_plan_part19.csv', '../data/train_plan_part9.csv', '../data/train_plan_part11.csv', '../data/train_plan_part1.csv', '../data/train_plan_part0.csv', '../data/train_plan_part18.csv', '../data/train_plan_part10.csv', '../data/train_plan_part12.csv', '../data/train_plan_part16.csv', '../data/train_plan_part15.csv', '../data/train_plan_part2.csv', '../data/train_plan_part14.csv', '../data/train_plan_part5.csv', '../data/train_plan_part7.csv', '../data/train_plan_part13.csv', '../data/train_plan_part4.csv']
总数据行数: 100000
df:
       id                                               json
0  85000  {"Plan": {"Node Type": "Bitmap Heap Scan", "Pa...
1  85001  {"Plan": {"Node Type": "Gather", "Parallel Awa...
2  85002  {"Plan": {"Node Type": "Hash Join", "Parallel ...
3  85003  {"Plan": {"Node Type": "Gather", "Parallel Awa...
4  85004  {"Plan": {"No

# 数据预处理
首先获取整个数据集的所有key值,以及对应key值的value值有哪些种类.     
然后根据key值的种类构建词表,用于最后编码.

我希望的vector组成:     
每个节点必有的:     
{'Plan Width', 'Node Type', 'Parallel Aware', 'Plan Rows'}

每个节点可能有的:







In [5]:
import sys
import os
import re
from typing import Dict, Any
sys.path.append(os.path.join(os.path.dirname('.'), '..'))

# Plan数据的部分检查函数
def check_relation_name_have_alias(dic: Dict[str, Any]) -> bool:
    if dic.get("Relation Name"):   # A 存在且非空
        if not dic.get("Alias"):   # B 不存在或为空
            print("Relation Name not have Alias")
            return False
    return True

def show_all_keys(plans_tree):
    keys_set = set()
    must_have_keys = None   # 用 None 表示初始值
    every_nodetype_keys = dict()
    
    for i in plans_tree:
        dic = i.getExtraInfo()

        if not check_relation_name_have_alias(dic):
            print("存在Relation Name没有Alias")
            assert False
        
        # 更新该节点类型对应的keys
        if dic.get("Node Type"):
            node_type = dic["Node Type"]
            if node_type not in every_nodetype_keys:
                every_nodetype_keys[node_type] = set(dic.keys())
            else:
                # every_nodetype_keys[node_type] &= set(dic.keys())
                every_nodetype_keys[node_type].update(dic.keys())

        # 更新所有 key（并集）
        keys_set.update(dic.keys())
        
        # 更新 must_have_keys（交集）
        if must_have_keys is None:
            must_have_keys = set(dic.keys())  # 第一个 dic 初始化
        else:
            must_have_keys &= set(dic.keys()) # 交集运算
    
    return keys_set, must_have_keys, every_nodetype_keys

def show_every_keys_value(plans_tree, keys_set):
    keys_value_dic = {k: set() for k in keys_set}
    for i in plans_tree:
        dic = i.getExtraInfo()
        for k in keys_set:
            if k in dic:
                keys_value_dic[k].add(dic[k])
    return keys_value_dic

def getPlanInfo(plans_tree):
    keys_set, must_have_keys, every_nodetype_keys = show_all_keys(plans_tree)
    print("keys_set:", len(keys_set), keys_set)
    print("must_have_keys:", len(must_have_keys), must_have_keys)
    for k, v in every_nodetype_keys.items():
        print(k, len(v), v)
        
    keys_value_dic = show_every_keys_value(plans_tree, keys_set)
    for k, v in keys_value_dic.items():
        if len(v) < 10:
            print(k, len(v), v)
        # 检查Recheck Cond和Index Cond是否存在非比较运算符
        elif k == "Recheck Cond" or k == "Index Cond" or k == "Filter":
            print(k, len(v), list(v)[:10])
            for i in list(v):
                if not re.search(r"[=><]", i):
                    print(i)
                    assert False
        else:
            print(k, len(v), list(v)[:10])

# 预处理数据
from models.DataPreprocessor import PlanNode, DataPreprocessor
preprocessor = DataPreprocessor()
plans_tree = preprocessor.preprocess_all(plans_dict)

# 展示
for i in range(5):
    print(plans_tree[i])
print("--------------------------------")
preprocessor.print_tree(plans_tree[0])
print("--------------------------------")
getPlanInfo(plans_tree)



# 统计信息
from models.Utils import load_column_stats, process_join_cond_field, process_index_cond_field
table_name_to_short_name = {'title': 't', 'movie_keyword': 'mk', 'movie_info_idx': 'mi_idx', 'movie_info': 'mi', 'movie_companies': 'mc', 'cast_info': 'ci'}
print("### 测试 load_column_stats ###")
col_vocab, scalers, col_stats_map = load_column_stats("../data/column_min_max_vals.csv")
print(col_vocab)
print(scalers)
print(col_stats_map)


PlanNode(node_type='Bitmap Heap Scan', children=[PlanNode(node_type='Bitmap Index Scan', children=[], extra_info={'Node Type': 'Bitmap Index Scan', 'Parent Relationship': 'Outer', 'Parallel Aware': False, 'Index Name': 'keyword_id_movie_keyword', 'Startup Cost': 0.0, 'Total Cost': 11531.73, 'Plan Rows': 1028173, 'Plan Width': 0, 'Actual Startup Time': 39.572, 'Actual Total Time': 39.572, 'Actual Rows': 1029758, 'Actual Loops': 1, 'Index Cond': '(keyword_id > 17243)'}, node_vector=None)], extra_info={'Node Type': 'Bitmap Heap Scan', 'Parallel Aware': False, 'Relation Name': 'movie_keyword', 'Alias': 'mk', 'Startup Cost': 11788.77, 'Total Cost': 49094.94, 'Plan Rows': 1028173, 'Plan Width': 12, 'Actual Startup Time': 41.924, 'Actual Total Time': 200.35, 'Actual Rows': 1029758, 'Actual Loops': 1, 'Recheck Cond': '(keyword_id > 17243)', 'Rows Removed by Index Recheck': 0, 'Exact Heap Blocks': 24037, 'Lossy Heap Blocks': 0}, node_vector=None)
PlanNode(node_type='Gather', children=[PlanNode(

In [3]:
process_join_cond_field("(t.id = mc.movie_id)")

['t.id', '=', 'mc.movie_id']

In [4]:
# 你已有的计划树
roots: List[PlanNode] = [...]  # 训练前全量或训练集子集

# (1) 离线准备
node_type_vocab = build_node_type_vocab(roots, add_unk=True)
stats_scalers   = build_stats_scalers(roots, stats_keys=['Plan Rows','Plan Width','Startup Cost','Total Cost'])

# (2) 载入 CSV 列统计（一次性）
col_vocab, col_scalers, col_stats_map = load_column_stats("/path/to/columns.csv")

# (3) 初始化编码器（固定词表，不在训练时增量）
encoder = NodeEncoder(
    node_type_vocab=node_type_vocab,
    stats_scalers=stats_scalers,
    col_vocab=col_vocab,
    col_scalers=col_scalers,
    col_stats_map=col_stats_map,
    operator_embedding_dim=32,
    stats_hidden_dim=16,
    predicate_dim=8,
    column_embedding_dim=16,
    column_stat_dim=8,
    output_dim=64
)

# (4) 编码
vec = encoder.encode_node(roots[0])   # [64]


NameError: name 'List' is not defined

In [ ]:
# NodeEncoder
from models.NodeEncoder import NodeEncoder
nodeEncoder = NodeEncoder()
from typing import Any, Dict, Iterable, List, Optional

nodeEncodedVectorsBox = []
for plan_tree in plans_tree:
    all_nodes = nodeEncoder.collect_nodes(plan_tree, method="dfs")
    nodeEncodedVectors = nodeEncoder.encode_nodes(all_nodes)
    nodeEncodedVectorsBox.append(nodeEncodedVectors)


print("nodeEncodedVectorsBox:", len(nodeEncodedVectorsBox))
# 查看第一个plan的编码向量
print(len(nodeEncodedVectorsBox[0]))
preprocessor.print_tree(plans_tree[0])

# 检查每个vector是否为空
for i in range(len(nodeEncodedVectorsBox)):
    for j in range(len(nodeEncodedVectorsBox[i])):
        if nodeEncodedVectorsBox[i][j] is None:
            print(f"nodeEncodedVectorsBox[{i}][{j}] is None")




nodeEncodedVectorsBox: 5000
5
└── Gather (Total Cost: 154548.95, Startup Cost: 23540.58, Plan Rows: 567655, Plan Width: 119, Actual Total Time: 646.97, Actual Rows: 283812), node_vector_shape: torch.Size([128])
    └── Hash Join (Total Cost: 96783.45, Startup Cost: 22540.58, Plan Rows: 236523, Plan Width: 119, Actual Total Time: 518.49, Actual Rows: 94604, Join Type: Inner), node_vector_shape: torch.Size([128])
        ├── Seq Scan (Total Cost: 49166.46, Startup Cost: 0.00, Plan Rows: 649574, Plan Width: 94, Actual Total Time: 147.05, Actual Rows: 514421, Relation Name: title, Alias: t), node_vector_shape: torch.Size([128])
        └── Hash (Total Cost: 15122.68, Startup Cost: 15122.68, Plan Rows: 383592, Plan Width: 25, Actual Total Time: 103.55, Actual Rows: 306703), node_vector_shape: torch.Size([128])
            └── Seq Scan (Total Cost: 15122.68, Startup Cost: 0.00, Plan Rows: 383592, Plan Width: 25, Actual Total Time: 54.38, Actual Rows: 306703, Relation Name: movie_info_idx, Al

In [ ]:
from models.TreeEncoder import GATTreeEncoder, TreeToGraphConverter
import torch

treeToGraphConverter = TreeToGraphConverter()
gatTreeEncoder = GATTreeEncoder(
    input_dim=64,      # 一定用实际特征维度
    hidden_dim=64,
    output_dim=64,
    num_layers=3,
    num_heads=4,
    dropout=0.1,
    pooling="mean"
)

planEmbeddingBox = []
for plan_tree in plans_tree:
    edge_index, x = treeToGraphConverter.tree_to_graph(plan_tree)
    x = torch.stack(
        [torch.as_tensor(f, dtype=torch.float32) for f in x],
        dim=0
    )
    planEmbeddingBox.append(gatTreeEncoder(x, edge_index))

print("x.shape, edge_index.shape:", x.shape, edge_index.shape)
for i in range(5):
    print(planEmbeddingBox[i].shape)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x128 and 64x256)

In [ ]:

# PredictionHead
from models.PredictionHead import PredictionHead
predictionHead = PredictionHead()

predictionBox = []
for planEmbedding in planEmbeddingBox:
    prediction = predictionHead.predict(planEmbedding)
    predictionBox.append(prediction)


print("predictionBox:", len(predictionBox))
for i in range(5):
    print(predictionBox[i], ExecutionTimes[i])



predictionBox: 5000
0.9999457877129316 654.241
4.700297597795725 349.797
-0.3459443412721157 1699.24
1.3503599683754146 345.056
1.109332099556923 90.666
